#### merge urls to recipe dataframe

In [2]:
# import relevant libraries
import pandas as pd
import numpy as np
from collections import Counter
import nltk 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import ast
import re
import math

In [3]:
# recipes dataframes
# there is a mismatch in df3_urls and df3_recipes 2430 vs 1022
df1 = pd.read_csv('../../ScrapeData/csv_recipes/recipe_df1.csv')
df2 = pd.read_csv('../../ScrapeData/csv_recipes/recipe_df2.csv')
df3 = pd.read_csv('../../ScrapeData/csv_recipes/recipe_df3.csv')
df4 = pd.read_csv('../../ScrapeData/csv_recipes/recipe_df4.csv')
df5 = pd.read_csv('../../ScrapeData/csv_recipes/recipe_df5.csv')
df6 = pd.read_csv('../../ScrapeData/csv_recipes/recipe_df6.csv')
df7 = pd.read_csv('../../ScrapeData/csv_recipes/recipe_df7.csv')
df8 = pd.read_csv('../../ScrapeData/csv_recipes/recipe_df8.csv')

In [4]:
# concat the dataframes
df_recipes = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8])
df_recipes = df_recipes.drop(['Unnamed: 0','id'], axis=1) #drop columns not required

In [5]:
df_recipes=df_recipes.reset_index(drop='index')

In [6]:
# url dataframes
df1_url = pd.read_csv('../../ScrapeData/csv_urls/urls_df1.csv')
df2_url = pd.read_csv('../../ScrapeData/csv_urls/urls_df2.csv')
df3_url = pd.read_csv('../../ScrapeData/csv_urls/urls_df3.csv')
df3_url = df3_url.iloc[:1022,:]
df4_url = pd.read_csv('../../ScrapeData/csv_urls/urls_df4.csv')
df5_url = pd.read_csv('../../ScrapeData/csv_urls/urls_df5.csv')
df6_url = pd.read_csv('../../ScrapeData/csv_urls/urls_df6.csv')

In [7]:
# concat the dataframes url and recipes
df_url = pd.concat([df1_url,df2_url, df3_url, df4_url,df5_url,df6_url])
df_url = df_url.drop(['Unnamed: 0'], axis=1) #drop columns not required

In [8]:
df_url = df_url.reset_index(drop='index')

In [9]:
# merge df_url and df_recipes
df = pd.concat([df_url,df_recipes],axis=1)

#### preprocess the dataframe

In [10]:
df['makeitagainscore'] = pd.to_numeric(df['makeitagainscore'].str.replace('%', ''))

In [11]:
# drop rows if there are null value records
df.dropna(axis=0, how='any', inplace=True,subset=['bestrating','ingredients',\
                            'makeitagainscore','preparation','rating','reviews','tags','title'])

In [12]:
df["ingredients_refined"] = ""
for index, row in df.iterrows():
    ingredients_items = ast.literal_eval(row["ingredients"])
    df.set_value(index,'ingredients_refined',ingredients_items)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


In [13]:
df["tags_refined"] = ""
for index, row in df.iterrows():
    tag_items = ast.literal_eval(row["tags"])
    df.set_value(index,'tags_refined',tag_items)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


#### extract ingrediants

In [14]:
# list of prepositions
prepositions = ["A", "An", "a", "about", "above", "above", "across", "additional","after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "plus", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thick", "thickly", "thin", "thinly", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]

In [15]:
# Common measurement words and hard-coded words.
measurement_words = ['pound',
 'cubes',
  'cups',
  'cut',
 'kilogram',
 'ounce','-ounce',
 'gram',
 'cup',
 'spoon',
 'quart',
 'teaspoon','teaspoons',
 'smidgen',
 'drop',
 'gallon',
 'dash',
 'handful',
 'scoop',
 'bowl',
 'inch',
 '-inch',
 'pinch',
 'liter',
 'milliliter',
 'tablespoon', 'tablespoons',
 'fluid',
 'bottle',
 'tad',
 'pint', 'Tbsp', 'tsp','lb']

In [16]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sschadt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
# create a list of stopwords, prepositions, and measurement words
filtered = nltk.corpus.stopwords.words('english')
for word in prepositions:
    filtered.append(word)
for word in measurement_words:
    filtered.append(word)

In [32]:
df["ingredient_list"] = ""
for index, row in df.iterrows():
    ingredients_items = ast.literal_eval(row["ingredients"])

    ingredients = []
    pattern = "([^\d\\.\\,\\\;\\*\\]\\[\\%\\½\\<\\>\\:\\°\\?\\�\\}\\{\)\(\"\/\s]+)"
    for ingredient in ingredients_items:
        p = re.compile(pattern)
        result = p.findall(ingredient) 
        ingredients.append(result)
        
        
    single_ingr = sum(ingredients, [])
    recipe_ingr = list(set(single_ingr))


    filtered_ingr = []
    for item in recipe_ingr:
        if item not in filtered:
            filtered_ingr.append(item)
            
    # Singularize the plural words, except words ending in 'us'. We lose
    #  's' from 'seedless', etc, but those don't matter.
    ingredients_singularized = [word[:-1] for word in filtered_ingr if word[-1:] == "s" and word[-2:] != "us"]
    ingredients_not_singular = [word for word in filtered_ingr if word[-1:] != "s" or word[-2:] == "us"]
    ingredients_all = ingredients_singularized + ingredients_not_singular
    
    # Add list to new field
    df.set_value(index,"ingredient_list",ingredients_all)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [33]:
#export the refined dataframe to a csv
df.to_csv('df_similarity.csv')

#### similarity script

In [20]:
#sample_str = [
#    "rice", "carrots", "shallots"
#]
sample_str = ["fruit", "avocado", "natural foods", "soup", "cuisine", "vegetarian food", "asparagus"]

In [40]:
print(type(df['ingredient_list'][55]))

<class 'list'>


In [37]:
counterA = Counter(sample_str)
counterB = Counter(df['ingredient_list'][55])
print(counterA)
print(counterB)

Counter({'fruit': 1, 'avocado': 1, 'natural foods': 1, 'soup': 1, 'cuisine': 1, 'vegetarian food': 1, 'asparagus': 1})
Counter({'pea': 1, 'floret': 1, 'chive': 1, 'green': 1, 'seedles': 1, 'ounce': 1, 'grain': 1, 'herb': 1, 'seed': 1, 'cucumber': 1, 'avocado': 1, 'pepita': 1, 'egg': 1, 'clove': 1, 'kosher': 1, 'sliced': 1, 'parsley': 1, 'rice': 1, 'mini': 1, 'ground': 1, 'sugar': 1, 'snap': 1, 'fresh': 1, 'garlic': 1, 'oil': 1, 'salt': 1, 'quinoa': 1, 'pepper': 1, 'tarragon': 1, 'lemon': 1, 'freshly': 1, 'divided': 1, 'pumpkin': 1, 'whole-milk': 1, 'halved': 1, 'black': 1, 'chopped': 1, 'finely': 1, 'mixed': 1, 'toasted': 1, 'olive': 1, 'large': 1, 'pitted': 1, 'cooked': 1, 'baby': 1, 'juice': 1, 'yogurt': 1, 'ripe': 1, 'farro': 1, 'broccoli': 1, 'brown': 1})


In [22]:
def counter_cosine_similarity(c1, c2):
    '''a function to calculate the cosine similarity between two lists'''
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)

In [38]:
print(counter_cosine_similarity(counterA, counterB))

0.05292561240249631


In [23]:
print(len(df))

11494


In [49]:
df['similarity_score']=""

for index, row in df.iterrows():
    try:
        counterB = Counter(row['ingredient_list'])
        score = counter_cosine_similarity(counterA,counterB)
        df.set_value(index,'similarity_score',score)
    except ZeroDivisionError:
        df.set_value(index,'similarity_score',0)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


In [50]:
type(df["similarity_score"][55])

numpy.int64

In [41]:
#results_all = df.sort_values(by=['similarity_score','rating','makeitagainscore'],ascending=[False,False,False])
results_all = df.sort_values(by="similarity_score",ascending=False)

results = results_all[:3]

In [48]:
results.head(0)

,batch,urls,bestrating,imagesrc,ingredients,makeitagainscore,preparation,rating,reviews,tags,title,ingredients_refined,tags_refined,ingredient_list,similarity_score


In [27]:
results = results[['urls','ingredients_refined','tags_refined','title','rating','preparation','similarity_score']]

In [28]:
results = results.reset_index(drop='index')

In [29]:
results.to_json(orient='index')

'{"0":{"urls":"https:\\/\\/www.epicurious.com\\/recipes\\/food\\/views\\/chilled-avocado-and-zucchini-soup-12507","ingredients_refined":["1 medium zucchini, cut into 1\\/2-inch slices","1 California avocado","1 cup buttermilk","3\\/4 cup ice cubes plus additional to thin the soup if desired","2 tablespoons fresh lemon juice","1\\/4 teaspoon ground cumin","1 1\\/2 teaspoons minced fresh chives"],"tags_refined":["Soup\\/Stew","Milk\\/Cream","Blender","Steam","Vegetarian","Quick & Easy","Avocado","Zucchini","Summer","Chill","Chive","Gourmet"],"title":"Chilled Avocado and Zucchini Soup ","rating":3.0,"preparation":"In a steamer set over boiling water steam the zucchini, covered, for 10 minutes, or until it is very tender. In a blender pure\\u00e9 the zucchini with the avocado, halved, peeled, and chopped, the buttermilk, 3\\/4 cup of the ice cubes, the lemon juice, and the cumin, adding enough of the additional ice cubes to reach the desired consistency. Divide the soup between 2 bowls and